# 📚 Workshop: RAG + LangChain + Streamlit

## ช่วงที่ 5: Wrap-up & Advanced Topics (3:30 – 4:00)

---

### 🎯 วัตถุประสงค์การเรียนรู้
- Recap pipeline: Load → Split → Embed → Store → Retrieve → LLM
- ปัญหาและการแก้ไข (context, latency, token cost)
- แนะนำการใช้ Qdrant / Pinecone สำหรับ production
- การเพิ่ม Features: Upload PDF, Multi-documents, Source citations
- กิจกรรมปิดท้าย: ให้ผู้เรียนปรับ UI หรือเพิ่มฟีเจอร์

---


## 🔄 Recap: RAG Pipeline

### Pipeline ที่เราได้สร้างขึ้น:

```mermaid
graph TD
    A[PDF Documents] --> B[Document Loader]
    B --> C[Text Splitter]
    C --> D[Embeddings]
    D --> E[Vector Store]
    
    F[User Question] --> G[Embeddings]
    G --> H[Vector Search]
    E --> H
    H --> I[Retriever]
    I --> J[Context Documents]
    J --> K[Prompt Template]
    K --> L[LLM]
    L --> M[Answer]
```

### ขั้นตอนที่เราได้เรียนรู้:

1. **📄 Document Loading** - โหลด PDF ด้วย PyPDFLoader
2. **✂️ Text Splitting** - แบ่งเอกสารด้วย RecursiveCharacterTextSplitter
3. **🔢 Embeddings** - สร้าง vectors ด้วย HuggingFaceEmbeddings
4. **🗄️ Vector Store** - เก็บ vectors ใน FAISS
5. **🔍 Retrieval** - ค้นหาเอกสารที่เกี่ยวข้อง
6. **🤖 LLM** - สร้างคำตอบด้วย Groq Llama
7. **🎨 UI** - แสดงผลด้วย Streamlit


## ⚠️ ปัญหาและการแก้ไข

### 1. **Context Window Limit**

**ปัญหา:** LLM มีข้อจำกัดในการประมวลผลข้อความยาว

**การแก้ไข:**
- ใช้ Text Splitter แบ่งเอกสารเป็น chunks เล็กๆ
- ปรับค่า `chunk_size` ให้เหมาะสม
- ใช้ `chunk_overlap` เพื่อไม่ให้ข้อมูลขาดหาย

```python
# ตัวอย่างการปรับแต่ง
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # ลดขนาด chunk
    chunk_overlap=200  # เพิ่ม overlap
)
```

### 2. **Latency (ความล่าช้า)**

**ปัญหา:** การตอบสนองช้า

**การแก้ไข:**
- ใช้ Vector Store ที่เร็ว (FAISS)
- ลดค่า `k` (จำนวนเอกสารที่ค้นหา)
- ใช้ LLM ที่เร็ว (Groq)
- Cache embeddings และ vector store

### 3. **Token Cost**

**ปัญหา:** ค่าใช้จ่ายสูงเมื่อใช้ LLM API

**การแก้ไข:**
- ใช้ LLM ที่ราคาถูก (Groq)
- ลดขนาด context ที่ส่งไป
- ใช้ local embeddings (HuggingFace)
- Optimize prompt template


## 🏭 Production Vector Stores

### เปรียบเทียบ Vector Stores:

| Vector Store | Pros | Cons | Use Case |
|--------------|------|------|----------|
| **FAISS** | เร็ว, ใช้หน่วยความจำน้อย | ไม่ persistent, ไม่มี API | Development, Demo |
| **Qdrant** | Persistent, API, Metadata filtering | ต้องติดตั้ง server | Production |
| **Pinecone** | Managed service, Scalable | เสียค่าใช้จ่าย | Enterprise |
| **Chroma** | ง่ายต่อการใช้งาน, Persistent | Performance ไม่ดีเท่า | Small projects |

### Qdrant สำหรับ Production:

```python
# ติดตั้ง Qdrant
pip install qdrant-client

# รัน Qdrant server
docker run -p 6333:6333 qdrant/qdrant

# ใช้ Qdrant ในโค้ด
from langchain_community.vectorstores import Qdrant

vectorstore = Qdrant.from_documents(
    documents=chunks,
    embedding=embeddings,
    collection_name="documents",
    url="http://localhost:6333"
)
```

### Pinecone สำหรับ Enterprise:

```python
# ติดตั้ง Pinecone
pip install pinecone-client

# ตั้งค่า Pinecone
import pinecone
pinecone.init(api_key="your-api-key", environment="your-environment")

# ใช้ Pinecone
from langchain_community.vectorstores import Pinecone

vectorstore = Pinecone.from_documents(
    documents=chunks,
    embedding=embeddings,
    index_name="documents"
)
```


## 🚀 Advanced Features

### 1. **Upload PDF แบบ Real-time**

```python
# เพิ่มใน Streamlit
uploaded_file = st.file_uploader("อัปโหลด PDF", type="pdf")

if uploaded_file:
    # บันทึกไฟล์ชั่วคราว
    with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as tmp_file:
        tmp_file.write(uploaded_file.getvalue())
        tmp_file_path = tmp_file.name
    
    # โหลดและประมวลผล PDF
    loader = PyPDFLoader(tmp_file_path)
    documents = loader.load()
    
    # สร้าง vector store ใหม่
    new_vectorstore = FAISS.from_documents(documents, embeddings)
    
    # อัปเดต RAG service
    rag_service.vectorstore = new_vectorstore
```

### 2. **Multi-documents Support**

```python
# รองรับหลายไฟล์
def load_multiple_documents(file_paths):
    all_documents = []
    
    for file_path in file_paths:
        if file_path.endswith('.pdf'):
            loader = PyPDFLoader(file_path)
            documents = loader.load()
            all_documents.extend(documents)
        elif file_path.endswith('.txt'):
            loader = TextLoader(file_path)
            documents = loader.load()
            all_documents.extend(documents)
    
    return all_documents

# สร้าง vector store จากหลายเอกสาร
documents = load_multiple_documents(['doc1.pdf', 'doc2.pdf', 'doc3.txt'])
vectorstore = FAISS.from_documents(documents, embeddings)
```

### 3. **Enhanced Source Citations**

```python
# ปรับปรุงการแสดงแหล่งอ้างอิง
def format_sources(sources):
    formatted_sources = []
    
    for i, source in enumerate(sources, 1):
        formatted_source = {
            "number": i,
            "content": source['content'][:300] + "...",
            "page": source['metadata'].get('page', 'N/A'),
            "file": source['metadata'].get('source', 'Unknown'),
            "relevance_score": source.get('score', 'N/A')
        }
        formatted_sources.append(formatted_source)
    
    return formatted_sources

# แสดงใน Streamlit
for source in formatted_sources:
    st.markdown(f"**แหล่งที่ {source['number']}:**")
    st.markdown(f"📄 ไฟล์: {source['file']}")
    st.markdown(f"📄 หน้า: {source['page']}")
    st.markdown(f"📊 คะแนนความเกี่ยวข้อง: {source['relevance_score']}")
    st.markdown(f"📝 เนื้อหา: {source['content']}")
    st.markdown("---")
```


## 🎯 กิจกรรมปิดท้าย

### ให้ผู้เรียนปรับ UI หรือเพิ่มฟีเจอร์:

#### 1. **ปรับปรุง UI**
- เปลี่ยนสีธีม
- เพิ่ม animations
- ปรับ layout
- เพิ่ม emojis

#### 2. **เพิ่มฟีเจอร์**
- Upload PDF แบบ real-time
- รองรับหลายไฟล์
- Export การสนทนา
- Search ในประวัติการสนทนา

#### 3. **ปรับปรุง RAG**
- เปลี่ยน embedding model
- ปรับแต่ง prompt template
- เพิ่ม metadata filtering
- ใช้ chain type อื่น

#### 4. **เพิ่มการตั้งค่า**
- ปรับ chunk size
- เปลี่ยน temperature
- เลือก LLM model
- ตั้งค่า k value


## 📝 สรุป Workshop ทั้งหมด

### ✅ Output หลังจบคอร์ส

ผู้เรียนสามารถ:

1. **เข้าใจหลักการของ RAG** 🧠
   - รู้จักปัญหาของ LLM
   - เข้าใจแนวคิด RAG
   - รู้จักสถาปัตยกรรม RAG

2. **ใช้ LangChain สร้าง RAG pipeline** 🔗
   - Document Loading
   - Text Splitting
   - Embeddings
   - Vector Store
   - Retrieval
   - LLM Integration

3. **Deploy chatbot พร้อม UI บน Streamlit** 🎨
   - สร้าง Chat Interface
   - แสดงประวัติการสนทนา
   - แสดงแหล่งอ้างอิง
   - การตั้งค่าต่างๆ

4. **ขยายต่อยอด** 🚀
   - รองรับหลายไฟล์
   - Vector DB ภายนอก
   - Advanced Features
   - Production Deployment

### 🎓 ไฟล์ที่สร้างขึ้นทั้งหมด:

1. `01_introduction_to_rag_langchain.ipynb` - แนะนำ RAG และ LangChain
2. `02_data_ingestion_vector_store.ipynb` - การเตรียมข้อมูลและ Vector Store
3. `03_build_rag_pipeline.ipynb` - สร้าง RAG Pipeline
4. `04_streamlit_ui.ipynb` - สร้าง Streamlit UI
5. `05_wrapup_advanced_topics.ipynb` - Advanced Topics และสรุป
6. `streamlit_app.py` - Streamlit Application หลัก

### 🏆 ความสำเร็จ:

- ✅ สร้าง RAG Chatbot ที่สมบูรณ์
- ✅ ใช้เทคโนโลยีที่ทันสมัย (LangChain, Groq, Streamlit)
- ✅ มี UI ที่ใช้งานง่าย
- ✅ รองรับภาษาไทย
- ✅ มีแหล่งอ้างอิงที่ชัดเจน

---

**🎉 ขอแสดงความยินดี! คุณได้สร้าง RAG Chatbot ที่สมบูรณ์แล้ว!**

**📚 ต่อไป: ลองปรับปรุงและเพิ่มฟีเจอร์ตามที่ต้องการ**
